# 第五章：Layer 1 水分子分析 — `WaterParser` 详解

本章拆解 `WaterParser` 模块的工作原理：
1. 如何从原子坐标推断水分子拓扑（O-H 配对）
2. 取向角 θ 的定义和计算
3. 单帧密度分布的计算步骤
4. 单帧取向加权密度
5. 吸附层角度概率密度函数（PDF）

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data_example" / "potential"
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ase.io

from src.structure.Analysis.WaterAnalysis._common import (
    _parse_abc_from_md_inp,
    _detect_low_high_interface_fractions,
)

# 读取第一帧
md_inp_path = DATA_DIR / "md.inp"
xyz_path    = DATA_DIR / "md-pos-1.xyz"

a_A, b_A, c_A = _parse_abc_from_md_inp(md_inp_path)
atoms = ase.io.read(str(xyz_path), index=0)
atoms.set_cell([a_A, b_A, c_A])
atoms.set_pbc([True, True, True])

print(f"读取第一帧：{len(atoms)} 个原子")

---

## 5.1 水分子拓扑构建

### 问题

XYZ 文件只有**原子位置**，没有任何关于化学键的信息。  
程序必须根据**原子距离**来判断哪个 H 属于哪个 O。

### 策略：O-H 截断距离

- 典型 O-H 键长约 **0.96 Å**
- 相邻水分子的 O-H 距离通常 > 1.5 Å
- 截断距离设为 **1.25 Å** — 凡是 O-H 距离 < 1.25 Å 的，认为属于同一个水分子

流程：
1. 找到所有 O 和 H 原子
2. 对每个 O，找距离 < 1.25 Å 的 H
3. 每个 O 配对 2 个最近的 H → 一个水分子

In [ ]:
from src.structure.utils import detect_water_molecule_indices, get_water_oxygen_indices_array
from src.structure.utils.config import DEFAULT_WATER_OH_CUTOFF_A

print(f"O-H 截断距离: {DEFAULT_WATER_OH_CUTOFF_A} Å")
print(f"（物理意义：O-H 共价键长约 0.96 Å，远小于 1.25 Å）")
print()

# 检测水分子拓扑
水分子索引 = detect_water_molecule_indices(atoms)
氧原子索引 = get_water_oxygen_indices_array(水分子索引).reshape(-1)

print(f"检测结果：")
print(f"  水分子数量: {len(水分子索引)}")
print(f"  氧原子数量: {len(氧原子索引)} (== 水分子数量 ✓)")
print(f"  水分子索引数组形状: {水分子索引.shape}")

In [ ]:
# 用 DataFrame 展示水分子索引
元素 = np.array(atoms.get_chemical_symbols())
坐标 = atoms.get_positions()

df = pd.DataFrame({
    '水分子编号': range(len(水分子索引)),
    'O 原子编号': 水分子索引[:, 0],
    'H1 原子编号': 水分子索引[:, 1],
    'H2 原子编号': 水分子索引[:, 2],
})

# 验证：O-H 距离是否确实 < 1.25 Å
OH1距离 = []
OH2距离 = []
for row in 水分子索引[:5]:  # 只检查前5个
    O_pos  = 坐标[row[0]]
    H1_pos = 坐标[row[1]]
    H2_pos = 坐标[row[2]]
    # 注意：需要用 MIC（最小镜像约定）来处理周期性边界
    d1 = atoms.get_distance(row[0], row[1], mic=True)
    d2 = atoms.get_distance(row[0], row[2], mic=True)
    OH1距离.append(d1)
    OH2距离.append(d2)

df_验证 = df.head(5).copy()
df_验证['O-H1 距离 (Å)'] = [f"{d:.4f}" for d in OH1距离]
df_验证['O-H2 距离 (Å)'] = [f"{d:.4f}" for d in OH2距离]

print("前 5 个水分子的原子索引和 O-H 距离验证：")
print(df_验证.to_string(index=False))
print(f"\n✅ 所有 O-H 距离均 < {DEFAULT_WATER_OH_CUTOFF_A} Å")

In [ ]:
# 可视化：水分子索引数组的彩色网格（前30个水分子）
展示数量 = min(30, len(水分子索引))
展示数组 = 水分子索引[:展示数量]

fig, ax = plt.subplots(figsize=(4, 9))
im = ax.imshow(展示数组, aspect='auto', cmap='YlOrRd')

ax.set_xticks([0, 1, 2])
ax.set_xticklabels(['O 编号', 'H1 编号', 'H2 编号'], fontsize=10)
ax.set_ylabel('水分子编号', fontsize=10)
ax.set_title(f'水分子索引数组\nshape={展示数组.shape}\n（前 {展示数量} 个水分子）', fontsize=11)

for i in range(展示数量):
    for j in range(3):
        ax.text(j, i, str(展示数组[i, j]), ha='center', va='center', fontsize=6)

plt.colorbar(im, ax=ax, label='原子编号', shrink=0.6)
plt.tight_layout()
plt.show()

---

## 5.2 取向角 θ 的定义

### 核心概念：H-O-H 角平分线

每个水分子的取向用一个**角平分线向量**（bisector）来描述：
- 角平分线是 O→H1 和 O→H2 方向的**归一化向量之和**
- 这个向量大致与水分子的偶极矩方向一致

**θ**（theta）定义为角平分线与 **+c 轴方向**的夹角：

```
    +c 方向（向上）
         ↑
         │  ← θ = 角平分线与 +c 的夹角
         │╲
         │ ╲  bisector（角平分线）
    O    │
   / \   │
  H   H  │

θ = 0°：  H 朝上（角平分线朝 +c），H 指向体相水方向
θ = 90°：  H 朝侧面（角平分线与 c 垂直）
θ = 180°：H 朝下（角平分线朝 -c），H 指向金属方向
```

In [ ]:
# 绘制水分子取向示意图
fig, axes = plt.subplots(1, 3, figsize=(12, 5))

for ax, (theta, H方向描述) in zip(axes, [
    (30,  'θ≈30°\nH 偏向上方（+c）'),
    (90,  'θ=90°\nH 在侧面'),
    (150, 'θ≈150°\nH 偏向下方（金属方向）'),
]):
    theta_rad = np.radians(theta)
    
    # 角平分线方向（从 O 出发）
    bisector = np.array([np.sin(theta_rad), np.cos(theta_rad)])  # (sin, cos) → x, y
    
    # H1, H2 位置（从 bisector 两侧各偏 52°，这是 H-O-H 角的一半）
    hoh_half = np.radians(52)
    h1_dir = np.array([np.sin(theta_rad + hoh_half), np.cos(theta_rad + hoh_half)])
    h2_dir = np.array([np.sin(theta_rad - hoh_half), np.cos(theta_rad - hoh_half)])
    
    O = np.array([0, 0])
    H1 = O + h1_dir * 0.96
    H2 = O + h2_dir * 0.96
    
    # 绘制原子
    ax.scatter(*O,  s=400, c='tab:red', zorder=5, label='O')
    ax.scatter(*H1, s=200, c='lightgray', zorder=5, label='H', edgecolors='black', linewidth=0.5)
    ax.scatter(*H2, s=200, c='lightgray', zorder=5, edgecolors='black', linewidth=0.5)
    
    # 绘制 O-H 键
    for H in [H1, H2]:
        ax.plot([O[0], H[0]], [O[1], H[1]], 'k-', linewidth=2, zorder=4)
    
    # 绘制角平分线（延伸）
    bv = bisector / np.linalg.norm(bisector)
    ax.annotate('', xy=O + bv * 1.3, xytext=O,
                arrowprops=dict(arrowstyle='->', color='tab:green', lw=2))
    ax.text(*(O + bv * 1.5), 'bisector', color='tab:green', ha='center', fontsize=8)
    
    # 绘制 +c 轴
    ax.annotate('', xy=(0, 1.6), xytext=(0, -1.6),
                arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))
    ax.text(0.05, 1.5, '+c', color='gray', fontsize=9)
    
    # 标注角度 θ
    角度弧 = np.linspace(np.pi/2, np.pi/2 - theta_rad, 30)
    ax.plot(0.4 * np.cos(角度弧), 0.4 * np.sin(角度弧), 'b-', linewidth=1.5)
    ax.text(0.55 * np.cos(np.pi/2 - theta_rad/2),
            0.55 * np.sin(np.pi/2 - theta_rad/2),
            f'θ={theta}°', color='blue', fontsize=9, ha='center')
    
    ax.set_xlim(-1.8, 1.8)
    ax.set_ylim(-1.8, 1.8)
    ax.set_aspect('equal')
    ax.set_title(H方向描述, fontsize=10)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.legend(loc='lower left', fontsize=8)

plt.suptitle('水分子取向角 θ 的定义\n（θ = bisector 与 +c 方向的夹角）', fontsize=12)
plt.tight_layout()
plt.show()

### 计算 cosθ

程序内部用 **cosθ** 而不是 θ 本身来工作（计算更高效），  
最后需要时再转回角度：`θ = arccos(cosθ)`

In [ ]:
from src.structure.utils.WaterParser import (
    _oxygen_to_hydrogen_map,
    _compute_bisector_cos_theta_vec,
)

# 构建 O → (H1, H2) 映射
o_to_h = _oxygen_to_hydrogen_map(水分子索引)

# 计算 +c 轴方向的单位向量
cell = atoms.cell.array
c_vec = cell[2]                         # 晶胞 c 向量
c_unit = c_vec / np.linalg.norm(c_vec)  # 归一化为单位向量

print(f"c 轴方向向量: {c_vec}")
print(f"c 单位向量: {c_unit}")
print(f"（正交晶胞：c 方向 = +z 方向 = (0, 0, 1)）")
print()

# 批量计算所有水分子的 cosθ
cos_theta_all = _compute_bisector_cos_theta_vec(atoms, 氧原子索引, o_to_h, c_unit)

print(f"cosθ 数组：")
print(f"  形状: {cos_theta_all.shape}  （每个水分子一个值）")
print(f"  范围: [{cos_theta_all.min():.3f}, {cos_theta_all.max():.3f}]  （cosθ 范围 -1 到 +1）")
print(f"  均值: {cos_theta_all.mean():.4f}  （接近0 → 平均取向接近随机）")
print(f"\n前 5 个水分子的 cosθ 和对应角度：")
for i in range(5):
    ct = cos_theta_all[i]
    theta = np.degrees(np.arccos(np.clip(ct, -1, 1)))
    print(f"  水分子 {i}: cosθ = {ct:+.4f}, θ = {theta:.1f}°")

---

## 5.3 单帧密度分布计算

### 步骤概览

1. 确定分析范围：从界面到晶胞中点
2. 把该范围分成小格（bins），每格宽度 0.1 Å
3. 统计每格中有多少个水分子的 O
4. 换算成质量密度（g/cm³）：`质量 / 体积`

In [ ]:
# 步骤1：确定分析范围
low_c, high_c = _detect_low_high_interface_fractions(atoms)

# 从 low_c 界面出发，到晶胞中点（两界面的中间）
半路径_c = 0.5 * (high_c - low_c)  # 分数坐标中的半路径长度
半路径_A = 半路径_c * c_A           # 转换为 Å

print(f"分析范围：")
print(f"  起点：low_c = {low_c:.4f}  (z = {low_c * c_A:.3f} Å)")
print(f"  中点：{low_c + 半路径_c:.4f} (z = {(low_c + 半路径_c) * c_A:.3f} Å)")
print(f"  路径长度：{半路径_A:.3f} Å")

In [ ]:
# 步骤2：计算每个水分子 O 距界面的距离
scaled = atoms.get_scaled_positions(wrap=True)  # 分数坐标，范围 [0, 1)
O_c_frac = scaled[氧原子索引, 2]               # O 原子的 c 分数坐标

# 相对于 low_c 界面的 c 分数距离
delta_c = np.mod(O_c_frac - low_c, 1.0)        # 对周期性取模，确保范围 [0, 1)
delta_A = delta_c * c_A                          # 转换为 Å

# 只保留半路径内的水分子（从界面到中点）
在分析范围内 = delta_A <= 半路径_A
选中_delta_A = delta_A[在分析范围内]

print(f"所有水分子 O 的距离分布：")
print(f"  O 总数: {len(O_c_frac)}")
print(f"  在分析范围内 (0 ~ {半路径_A:.1f} Å): {在分析范围内.sum()} 个")
print(f"  （另一半在 high_c 界面附近，用于对称性平均）")

In [ ]:
# 步骤3：分箱统计（histogram）
from src.structure.utils.config import DEFAULT_Z_BIN_WIDTH_A

dz_A = DEFAULT_Z_BIN_WIDTH_A  # 0.1 Å

# 建立分箱边界
nbins = max(1, int(np.ceil(半路径_A / dz_A)))
bin_edges = np.concatenate(([0.0], np.arange(1, nbins) * dz_A, [半路径_A]))
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

# 统计每个箱中的水分子数
counts, _ = np.histogram(选中_delta_A, bins=bin_edges)

print(f"分箱参数：")
print(f"  bin 宽度 (dz_A): {dz_A} Å")
print(f"  bin 总数: {nbins}")
print(f"  bin 中心范围: {bin_centers[0]:.3f} ~ {bin_centers[-1]:.3f} Å")
print(f"\n前 10 个 bin 的水分子数：")
for i in range(10):
    print(f"  bin {i}: 中心 {bin_centers[i]:.2f} Å, 水分子数 {counts[i]}")

In [ ]:
# 步骤4：换算为质量密度 (g/cm³)
# 公式：密度 = 质量 / 体积
# 每格的质量 = 格内水分子数 × 单个水分子质量
# 每格的体积 = xy 截面积 × bin 宽度

AVOGADRO = 6.02214076e23  # 阿伏伽德罗常数 (mol⁻¹)
ANGSTROM3_TO_CM3 = 1.0e-24  # Å³ 转 cm³
WATER_MOLAR_MASS = 18.01528  # g/mol

单个水分子质量_g = WATER_MOLAR_MASS / AVOGADRO  # 约 2.99e-23 g

# xy 截面积
cell_matrix = atoms.cell.array
area_xy_A2 = float(np.linalg.norm(np.cross(cell_matrix[0], cell_matrix[1])))

bin_widths_A = np.diff(bin_edges)           # 每个 bin 的宽度（Å）
bin_volumes_A3 = area_xy_A2 * bin_widths_A  # 每个 bin 的体积（Å³）

mass_per_bin_g = counts * 单个水分子质量_g  # 每个 bin 的总质量（g）
rho_g_cm3 = mass_per_bin_g / (bin_volumes_A3 * ANGSTROM3_TO_CM3)  # 密度（g/cm³）

print(f"物理常数：")
print(f"  单个水分子质量: {单个水分子质量_g:.4e} g")
print(f"  xy 截面积: {area_xy_A2:.3f} Å²")
print(f"  1 个 bin 的体积: {bin_volumes_A3[0]:.4f} Å³")
print(f"\n密度结果：")
print(f"  密度范围: {rho_g_cm3.min():.3f} ~ {rho_g_cm3.max():.3f} g/cm³")
print(f"  （体相水参考: ~1.0 g/cm³）")

In [ ]:
# 可视化：单帧密度分布
fig, ax = plt.subplots(figsize=(9, 4))

ax.plot(bin_centers, rho_g_cm3, color='tab:blue', linewidth=1.2, alpha=0.8, label='单帧密度（第一帧）')
ax.axhline(y=1.0, color='gray', linestyle='--', linewidth=1, label='体相水参考值 1.0 g/cm³')

# 标注第一个峰（吸附层）
第一峰索引 = np.argmax(rho_g_cm3)
ax.axvline(x=bin_centers[第一峰索引], color='red', linestyle=':', linewidth=1.5,
           label=f'密度最大值位置 ({bin_centers[第一峰索引]:.2f} Å)')

ax.set_xlabel('距 low_c 界面的距离 (Å)', fontsize=12)
ax.set_ylabel('水密度 (g/cm³)', fontsize=12)
ax.set_title('单帧（第一帧）水的质量密度分布', fontsize=13)
ax.legend(fontsize=10)
ax.set_xlim(0, None)

plt.tight_layout()
plt.show()

print("注意：单帧结果噪声较大，第六章会展示如何通过多帧平均来平滑曲线。")

---

## 5.4 单帧取向加权密度

**取向加权密度**是对密度的修正：每个水分子的贡献乘以其 cosθ，  
而不是简单地计数 +1。

公式：
$$\rho_{orient}(z) = \frac{\sum_{i \in bin(z)} \cos\theta_i \cdot m_{H_2O}}{V_{bin}}$$

其中 $m_{H_2O}$ 是单个水分子的质量，$V_{bin}$ 是 bin 体积。

In [ ]:
# 获取分析范围内的 cosθ 值
选中_O索引 = 氧原子索引[在分析范围内]
选中_cos_theta = cos_theta_all[在分析范围内]

# 加权直方图：每个箱的 cosθ 之和
cos_sum_per_bin, _ = np.histogram(选中_delta_A, bins=bin_edges, weights=选中_cos_theta)

# 换算为取向加权密度
orient_density = cos_sum_per_bin * 单个水分子质量_g / (bin_volumes_A3 * ANGSTROM3_TO_CM3)

print(f"取向加权密度范围: {orient_density.min():.3f} ~ {orient_density.max():.3f} g/cm³")

# 对比密度和取向加权密度
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(9, 7), sharex=True)

ax1.plot(bin_centers, rho_g_cm3, color='tab:blue', linewidth=1.2, label='质量密度 ρ')
ax1.axhline(y=1.0, color='gray', linestyle='--', linewidth=1)
ax1.set_ylabel('密度 (g/cm³)', fontsize=11)
ax1.set_title('质量密度 vs 取向加权密度（第一帧）', fontsize=12)
ax1.legend(fontsize=10)

ax2.plot(bin_centers, orient_density, color='tab:orange', linewidth=1.2, label='取向加权密度 ρ·cosθ')
ax2.axhline(y=0, color='gray', linestyle='--', linewidth=1)
ax2.set_xlabel('距界面距离 (Å)', fontsize=11)
ax2.set_ylabel('取向密度 (g/cm³)', fontsize=11)
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

print("\n物理解读：")
print("  取向加权密度 > 0 → 该区域水分子平均 H 朝向 +c（朝上，背离金属）")
print("  取向加权密度 < 0 → 该区域水分子平均 H 朝向 -c（朝下，指向金属）")
print("  值接近0 → 水分子取向随机，无定向")

---

## 5.5 吸附层角度分布（PDF）

**吸附层**（adsorbed layer）是最靠近金属表面的一层水分子。  
对吸附层水分子单独统计 θ 的分布，可以揭示它们的偏好取向。

In [ ]:
# 确定吸附层范围（简单方法：取密度最高的第一个峰附近）
# 这里用一个简单的固定范围来演示：0 ~ 4 Å
吸附层范围_A = (0.0, 4.0)

# 筛选在吸附层内的水分子
在吸附层 = (选中_delta_A >= 吸附层范围_A[0]) & (选中_delta_A <= 吸附层范围_A[1])
吸附层_cos_theta = 选中_cos_theta[在吸附层]

# 将 cosθ 转换为角度 θ（0°~180°）
cos_clipped = np.clip(吸附层_cos_theta, -1.0, 1.0)
theta_deg = np.degrees(np.arccos(cos_clipped))

print(f"吸附层（0 ~ {吸附层范围_A[1]} Å）内的水分子：")
print(f"  数量: {len(theta_deg)} 个")
print(f"  θ 范围: {theta_deg.min():.1f}° ~ {theta_deg.max():.1f}°")
print(f"  平均 θ: {theta_deg.mean():.1f}°")

In [ ]:
# 计算概率密度函数（PDF）
from src.structure.utils.config import DEFAULT_THETA_BIN_DEG

ndeg = DEFAULT_THETA_BIN_DEG  # 5.0°
n_theta_bins = int(180 / ndeg)
theta_edges = np.linspace(0.0, 180.0, n_theta_bins + 1)
theta_centers = 0.5 * (theta_edges[:-1] + theta_edges[1:])

# density=True 使得结果是概率密度（积分 = 1）
theta_pdf, _ = np.histogram(theta_deg, bins=theta_edges, density=True)

fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(theta_centers, theta_pdf, color='tab:green', linewidth=1.5, marker='o', markersize=3)
ax.fill_between(theta_centers, theta_pdf, alpha=0.3, color='tab:green')

# 标注关键角度
ax.axvline(x=0,   color='blue', linestyle=':', linewidth=1, alpha=0.5)
ax.axvline(x=90,  color='gray', linestyle=':', linewidth=1, alpha=0.5)
ax.axvline(x=180, color='red',  linestyle=':', linewidth=1, alpha=0.5)

ax.text(5,   max(theta_pdf) * 0.9, 'θ=0°\nH朝上', ha='left', fontsize=8, color='blue')
ax.text(90,  max(theta_pdf) * 0.9, 'θ=90°\nH朝侧', ha='center', fontsize=8, color='gray')
ax.text(175, max(theta_pdf) * 0.9, 'θ=180°\nH朝下', ha='right', fontsize=8, color='red')

ax.set_xlabel('取向角 θ (度)', fontsize=12)
ax.set_ylabel('概率密度 (1/度)', fontsize=12)
ax.set_title(f'吸附层水分子取向角 PDF（第一帧，bin={ndeg}°）', fontsize=13)
ax.set_xlim(0, 180)

plt.tight_layout()
plt.show()

print(f"\n注意：单帧 PDF 非常嘈杂（只有 {len(theta_deg)} 个数据点）")
print(f"第六章会展示所有帧叠加后的平滑 PDF。")

---

## 本章小结

| 步骤 | 内容 | 关键函数/参数 |
|------|------|---------------|
| 水分子拓扑 | 根据 O-H 距离（< 1.25 Å）配对 | `detect_water_molecule_indices` |
| 取向角 θ | bisector（角平分线）与 +c 的夹角 | `_compute_bisector_cos_theta_vec` |
| 密度分布 | 水分子 O 的 z 坐标直方图 → 换算 g/cm³ | `np.histogram`, `bin_volumes` |
| 取向加权密度 | 加权直方图（权重 = cosθ）→ 换算 | 同上，加 `weights=cos_theta` |
| 取向角 PDF | 吸附层内 θ 的频率直方图（归一化） | `np.histogram(density=True)` |

**关键公式**：
$$\rho(z) = \frac{N_{bin}(z) \cdot m_{H_2O}}{V_{bin}}$$
$$\rho_{orient}(z) = \frac{\sum_{i \in bin} \cos\theta_i \cdot m_{H_2O}}{V_{bin}}$$

**下一章**（`06_layer2_trajectory_analysis.ipynb`）将展示如何对 274 帧进行集成平均，得到统计可靠的结果。